## Count words in purpose/design/findings/originality

In [1]:
import os
import pandas as pd
from transformers import RobertaTokenizer, RobertaTokenizerFast
# base_dir = '/home/ubuntu/efs/emerald/data/fulltext-abstract_sections/'
# wc = []
# for fname in os.listdir(base_dir):
#     if fname.endswith('.target') and 'bpe' not in fname:
#         with open(base_dir + fname) as f:
#             for line in f:
#                 wc.append(line.count(' ') + 1)
# wc_purpose, wc_method, wc_finding, wc_origin = zip(*[(wc[i*4], wc[i*4+1], wc[i*4+2], wc[i*4+3]) for i in range(len(wc)//4)])

# dfwc = pd.DataFrame({'word_count': wc})
# dfwc_sections = pd.DataFrame({'purpose': wc_purpose, 'method': wc_method, 'finding': wc_finding, 'origin': wc_origin, })
# dfwc_sections.describe()
# dfwc.describe()

In [2]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")


### from emerald_with_structured_abstr.jsonl

In [3]:
import json
jsonl = []
with open('/mnt/efs/emerald/emerald_with_structured_abstr.jsonl') as f:
    for line in f:
        jsonl.append(json.loads(line.strip()))
df = pd.DataFrame({'jsonl': jsonl})


In [1]:
len(df)
# dforg=df
# df = df.head()

NameError: name 'df' is not defined

In [5]:
def fulltext_length(obj):
    text = ' '.join(i for i in obj['section_names'] if i != '__NO_TITLE__') + ' ' + ' '.join(par for sec in obj['sections'] for par in sec)
#     print(text.count(' '))
#     print(len(tokenizer.tokenize(text)))
#     print("---")
    return len(tokenizer.tokenize(text))

df['fulltext_length'] = df.jsonl.apply(fulltext_length)

Token indices sequence length is longer than the specified maximum sequence length for this model (8101 > 512). Running this sequence through the model will result in indexing errors


In [6]:
def abstract_length(j):
    abstract_sections_names = j['abstract_sections_names']
    abstract_sections = j['abstract_sections']

    section_length = {}
    for title, sec in zip(abstract_sections_names, abstract_sections):
        if title not in ('Purpose', 'Design/methodology/approach', 'Findings', 'Originality/value'):
            continue
        text = ' '.join(par for par in sec)
#         print(text.count(' '))
#         print(len(tokenizer.tokenize(text)))
#         print("---")

        section_length[title] = len(tokenizer.tokenize(text))
        
    return [section_length[i] for i in ('Purpose', 'Design/methodology/approach', 'Findings', 'Originality/value')]

df['abstract_length'] = df.jsonl.apply(abstract_length)

In [7]:
df['ab_len_purpose'], df['ab_len_design'], df['ab_len_findings'], df['ab_len_origin'] = zip(*df.abstract_length)
df['abstract_length'] = df['abstract_length'].apply(sum)

In [8]:
df.describe(percentiles=[.01, .25, .5, .75, .99])

,fulltext_length,abstract_length,ab_len_purpose,ab_len_design,ab_len_findings,ab_len_origin
count,60146.000000,60146.000000,60146.000000,60146.000000,60146.000000,60146.000000
mean,7238.830047,234.698750,56.844628,55.526486,72.685848,49.641788
std,2871.696215,73.874763,30.631233,30.713551,36.095541,25.965273
min,77.000000,24.000000,7.000000,2.000000,2.000000,2.000000
1%,1185.000000,95.000000,17.000000,10.000000,18.000000,13.000000
25%,5517.000000,187.000000,34.000000,34.000000,48.000000,31.000000
50%,7188.000000,227.000000,50.000000,50.000000,67.000000,44.000000
75%,8870.000000,273.000000,72.000000,71.000000,91.000000,62.000000
99%,15058.750000,458.000000,153.000000,150.000000,185.000000,133.000000
max,88942.000000,2652.000000,679.000000,804.000000,1198.000000,497.000000


In [9]:
df[df.abstract_length < 50].shape

(13, 7)

In [10]:
j = df[df.abstract_length < 50]['jsonl'].tolist()[0]
print(j['journal'], '|||', j['title'])

Journal of Business Strategy ||| Paying attention and getting attention: two sides of the communication coin


In [11]:
from pprint import pprint
# pprint(j)

In [12]:
df[df.abstract_length > 500].shape

(295, 7)

In [13]:
j = df[df.abstract_length > 1000]['jsonl'].tolist()[0]
print(j['journal'], '|||', j['title'])

International Hospitality Review ||| Status and scope of project management in the hospitality industry


In [14]:
# pprint(j)

### Section titles in fulltext
* how many papers with intro section
* how many papers with method section
* ...

In [15]:
# ('Purpose', 'Design/methodology/approach', 'Findings', 'Originality/value')
def find_section_title_like(j, ts):
    section_names = j['section_names']
    for sn in section_names:
        for i in ts:
            if i.lower() in sn.lower():
                return 1
    return 0

df['has_intro'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['intro']))
df['has_purpose'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['purpose']))
df['has_design'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['design', 'method', 'approach']))
df['has_result'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['result', 'find']))
df['has_origin'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['origin', 'value']))
df['has_conclu'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['conclu']))

In [16]:
df.head(2)

,jsonl,fulltext_length,abstract_length,ab_len_purpose,ab_len_design,ab_len_findings,ab_len_origin,has_intro,has_purpose,has_design,has_result,has_origin,has_conclu
0,{'title': 'From the informal economy to the me...,8101,196,61,47,59,29,1,0,0,0,1,1
1,{'title': 'How close are formal and informal w...,7333,263,128,30,78,27,1,0,0,0,0,0


In [17]:
tmp = df[[i for i in df.columns if i.startswith('has_')]].agg(['sum'])
tmp

,has_intro,has_purpose,has_design,has_result,has_origin,has_conclu
sum,50451,641,40285,33872,1450,44797


In [18]:
# percent
tmp / df.shape[0]

,has_intro,has_purpose,has_design,has_result,has_origin,has_conclu
sum,0.838809,0.010657,0.669787,0.563163,0.024108,0.744804


In [19]:
# df.jsonl.apply(lambda x: find_section_title_like(x, ['introduction'])).sum()